# Gene Expression Analysis of Alzheimer's Disease
    With Neurosynth, AHBA, and Neuromaps
    Amy Hardy | COGS 199 | Spring 2022

## Background

### Alzheimer's Disease

Alzheimer's disease (AD), is a neurodegenerative disease that is characterized the presence of beta-amyloid plaques between neurons and tau tangles within neurons. However, whether these physiological components involved with AD are causative or consequential is unknown. AD is most commonly diagnosed by means of attention, memory, and language assessments, along with brain scans (computed tomography, magnetic resonance imaging, or positron emission tomography). Some of the most common symptoms of the disease include difficulty performing familiar tasks, aggitation, decreased judgement and attention skills, and depression. Being progressive in nature, AD typically starts with mild symptoms and develops towards its most serve stages in approximately 8-10 years.

The genetic basis of Alzheimer's disease is still widely unknown. This is true especially for sporadic AD, which accounts for about 95% of cases. The APOE-e4 gene widely known for being linked to AD, but there is not a causal link between it and having the disease. While having one or two of the e4 alleles does indeed increase your risk for developing AD, is it not guaranteed to occur. As for the remaining 5% of cases, familial AD is highly linked with mutations of the following the genes: APP, PSEN1, and PSEN2.  

Over recent years, the ongoing search for causes, biomarkers, and potential treatments have resulted in diverging opinions on it's pathology. "Reimagining Alzheimer’s Disease - An Age-Based Hypothesis" argues that the amyloid cascade hypothesis (idea that the neurodegeneration in AD is caused by abnormal accumulation of amyloid beta plaques) may not be accurate. Herrup proposes a new set of steps towards AD that starts with the risk factor, age, and is accelerated by some type of initiating injury, where microglial activation and inflammatory response play a major role.

With further advancement in the fields of genomics and a growing access to neural datasets, it is important to examine (gene-to-disease area), (gene-to-gene) and (gene-to-other-biological-ontology) statistical relationships in order to develop a well-rounded understanding of the brain's structure and function. 

#### Allen Human Brain Atlas Data

The Allen Human Brain Atlas (AHBA) is a freely available dataset containing microarray expression data collected from the human brain. "Derived from bulk microarray analysis of tissue samples obtained from six donors, the AHBA provides expression data for more than 20,000 genes across 3,702 brain areas in MRI-derived stereo- tactic space" [12].

* <u>abagen: A Python toolbox for the Allen Brain Atlas genetics data</u> <br>

    abagen provides access to AHBA with functionality to parcellate data into regions of interest, allowing a variety of options for probe selection, donor aggregation and data normalization. The goal of abagen is to provide a "more standardized and systematic research in imaging transcriptomics, and will help to advance future understanding of the influence of gene expression in the human brain" [12]. <br> 

    The first step with abagen in this analysis is to access an acceptable parcellation atlas, which is easily completed in this case with the <code>abagen.fetch_desikan_killiany()</code>. This returns a dictionary with keys: 'image': an atlas parcellation image to be used when fetching expression data, and 'info': a csv file with parcel id(1-83), label(brain area), hemisphere(L/R), and structure(cortex, subcortex/brainstem).

In [25]:
import abagen
import pandas as pd
atlas = abagen.fetch_desikan_killiany()
desikan_info = pd.read_csv(atlas['info'])
desikan_info[:5]

,id,label,hemisphere,structure
0,1,bankssts,L,cortex
1,2,caudalanteriorcingulate,L,cortex
2,3,caudalmiddlefrontal,L,cortex
3,4,cuneus,L,cortex
4,5,entorhinal,L,cortex


#### Neurosynth Data

Neurosynth, a platform for automatic meta-analysis, synthesizes data from approximately 15,000 neuroimaging studies that are published with corresponding fMRI voxel coordinates and provides probabilistic measures of a particular voxel being associated with a given term. In this analysis, we downloaded the association maps for the term "Alzheimer". There were 267 published studies that Neurosynth identified as being associated with Alzheimer's disease. 

In [2]:
import nibabel as nib
AD_association = nib.load('data/neurosynth_maps/terms/alzheimer_association-test_z_FDR_0.01.nii')


#### Neuromaps Parcellator
In order to compare AHBA with the Neurosynth map, they must be datasets of similar sizes (i.e. number of rows), so we will utilize a secondary parcellator method provided by **Neuromaps** (more on this later). Similarly to the AHBA data, the Neurosynth AD map will be parcellated with the Desikan Killany atlas.

In [ ]:
from neuromaps.parcellate import Parcellater
parc = Parcellater(atlas['image'], 'mni152')
AD_parc = parc.fit_transform(AD_association, 'mni152')

![image_name](./data/plots/dk_AD_plot.png)

## Gene Analysis

### 1. Over and under-expressed genes in five highest AD regions

The five rows (regions from Desikan Killany atlas) with the highest corresponding Alzheimer's association from the Neurosynth map were chosen and identified as the entorhinal cortex, hippocampus, inferior temporal gyrus, parahippocampal gyrus, and fusiform gyrus. Each of these defined areas seem to play a role in AD. The **entorhinal cortex** acts as a gateway for input and output to the hippocampus and expression of new memory relies on this entorhinal–hippocampal connectivity. In severe AD, the number of neurons in layer II of the entorhinal cortex is reduced by about 90% [4]. The **hippocampus** is known to be highly involved in learning and memory, and it is known to be affected in AD [6,7,8]. The inferior temporal gyrus, parahippocampal gyrus, and fusiform gyrus have also been linked to Alzheimer's and are some of the first areas to experience synaptic loss during the course of the disease[9,10]. Atrophy in the parahippocampal gyrus, may be a very important biomarker in detecting early AD [11].

<div>
<img src="./data/plots/over_under_ad_regions.png" width="900"/>
</div>

### 2. AD association -- gene expression correlation

<div>
<img src="./data/plots/ten_corr_subplots.png" width="850"/>
</div>

#### The top ten AD correlated genes are plotted above, in addition to their AD correlation bar plot on the bottom right. The bottom left plot shows the histogram for log transformed AD correlation across all genes which shows the correlation of genes is approximately normally distributed. 

<div>
<img src="./data/plots/hist_and_bar.png" width="850"/>
</div>

Another useful perspective in genetic analysis of diseases is to examine gene co-expression. "Recently, the weighted gene co-expression network analysis (WGCNA) has been widely used to identify clusters of co-expressed genes with highly correlated expression patterns based on the genetic profile of many diseases" Constructing a co-expression network and extracting hub genes following gene enrichment analysis recently identified four novel genes associated with AD [5]. All of the genes in the matrix below have a moderate to strong correlation with  Neurosynth derived AD association, so it is worth noting which of them are possibly being co-expressed with one another. 

Some of the significant results were: GADD45B & DCAF5, LANCL2 & DCAF5, NEURL1B & TRIM66.

<div>
<img src="./data/plots/ten_coexpression.png" width="700"/>
</div>

### 3. Guilt by association genes

As mentioned, APOE, APP, PSEN1, and PSEN2 are genes that are known to be involved in Alzheimer's disease. For this section, we plotted the genes most correlated with these four 'AD' genes. These were collected from a pairwise correlation matrix for all ~15,000 genes in the AHBA dataset. "Potential disease genes can be identified using a guilt-by-association (GBA) approach that highlights genes that are co-expressed with multiple disease genes" [2].

<div>
<img src="./data/plots/adgenescattercorr.png" width="900"/>
</div>

#### Lastly in the correlation analysis is these four 'AD' genes pairwise correlation with eachother.  While none of these correlations are very high, the largest r value seems to be between APOE and PSEN1.

<div>
<img src="./data/plots/adgene4.png" width="550"/>
</div>

## Neuromaps

Neuromaps is a toolbox designed to create a standardized workflow for contextualzing brain maps with the broader literature. The main functionality of Neuromaps is:

* Access to curated repository of annotations from recent literature 
* High-quality transformations between four standard coordinate systems
* Uniform interfaces for statistical comparisons between brain maps

After obtaining the list of ten genes most correlated with AD association, we obtained gene maps, which are derived from the AHBA dataset via <a href='https://neurosynth.org/genes/'>Neurosynth/Genes</a>. These maps transformed to surface space and were then plotted with <code>neuromaps.plotting.plot_surf_template()</code>.

<div>
<title>Top AD Associated Genes</title><br>
<img src="./data/plots/geneplots2.png" width="650"/>
</div>

#### MEG Oscillation Map to Gene Correlation

In [27]:
import nibabel as nib
from neuromaps import nulls
from neuromaps import stats
from neuromaps import transforms
from neuromaps import datasets
from neuromaps.datasets import available_tags, available_annotations, fetch_annotation

Neuromaps methods <code>available_tags()</code> and <code>available_annotations()</code> can be used to find annotations within their repository that best match a specific interest. In this case, we were interested in determining if any of the brain annotations measuring neural oscillations. 

In [30]:
for annotation in available_annotations(tags=['MEG'])[:-1]:
    print(annotation)

('hcps1200', 'megalpha', 'fsLR', '4k')
('hcps1200', 'megbeta', 'fsLR', '4k')
('hcps1200', 'megdelta', 'fsLR', '4k')
('hcps1200', 'meggamma1', 'fsLR', '4k')
('hcps1200', 'meggamma2', 'fsLR', '4k')
('hcps1200', 'megtheta', 'fsLR', '4k')



<div>
<img src="./data/plots/megmap.png" width="750"/>
</div>

Each of the above annotations were fetched from the Neuromaps data and we calculated the pairwise correlation between each of them and each of the top AD associated genes. As seen here, both maps in the correlation were transformed/resampled to an fsLR coordinate system with 32k resolution. Note that the MEG maps were already in an fsLR coordinate system, but Neuromaps does not currently support transforming into 4k resolution. The correlation analysis here includes spatial nulls for significance testing.

<code>
def spatial_nulls(map1, map2):
    fsav_map1 = transforms.fslr_to_fslr(map1, '32k')
    fsav_map2 = transforms.fslr_to_fslr(map2, '32k')
    rotated = nulls.alexander_bloch(fsav_map1, atlas='fsLR', density='32k', n_perm=100, 
    seed=1234)
    corr, pval = stats.compare_images(fsav_map1, fsav_map2, nulls=rotated)
    return (f'r = {corr:.3f}, p = {pval:.3f}') </code>

Results for these pairwise correlations were generally low, with the highest values being around r = 0.3. 

Therefore, they are excluded from this analysis for now.

However, when doing pairwise correlation between the MEG maps alone, there were more significant results:

    alpha & delta: r = -0.838 
    alpha & gamma1: r = -0.880
    alpha & theta: r = -0.886
    delta & gamma2: r = 0.843

        all with p-value < 0.05

See [genes_meg_oscillation_maps.ipynb](./B.ipynb) for detailed correlations.

#### Receptor Density Map to Gene Correlation

Similar to the process used for the MEG maps, the annotations seen in <code>data_desc</code> were fetched from the Neurosynth data and correlation between each of them and each gene was calculated. However, this time both maps were transformed to a fsaverage coordinate system with 10k resolution.

In [32]:
import json
with open('data/data_desc.json', 'r') as f:
    data_desc = json.load(f)
data_desc

{'dopamine': {'sandiego2015': 'D2 dopamine receptors',
  'sasaki2012': 'dopamine transporter',
  'kaller2017': 'dopamine D1 receptors',
  'alarkurtti2015': 'raclopride - dopamine receptor'},
 'gaba': {'norgaard2020': 'GABAA receptors',
  'dukart2018': 'flumazenil gaba anatgonist'},
 'norepinephrine': {'hesse2017': 'noradrenaline transporter'},
 'acetylcholine': {'tuominen': 'acetylcholine transporter density',
  'hillmer2016': 'nicotinic acetylcholine receptors',
  'naganawa2020': 'M1 muscarinic acetylcholine receptors'},
 'serotonin': {'gallezot2010': 'serotonin 5-HT(1B) receptor',
  'fazio2016': 'serotonin transporter',
  'beliveau2017': 'seratonin agonist',
  'radnakrishnan2018': '5-HT6 receptor availability'},
 'glutamate': {'rosaneto': 'mGluR5 ligand', 'smart2019': 'mGluR5'},
 'mu opioid': {'turtonen2020': 'carfentanil: mu opioid receptor availability',
  'kantonen2020': 'carfentanil: mu opioid receptor availability'}}


<div>
<img src="./data/plots/receptormaps.png" width="650"/>
</div>

See [genes_receptors.ipynb](./B.ipynb) for detailed correlations.

## Conclusion

As an extremely complicated neurodegenerative disease with no known cause or cure, Alzheimer's disease will continue to require a multitute of techniques of analysis and investigation. Making use of online datasets and toolboxes allows us to make inferences about possible connections and perform statistical analysis. Going forward, I plan to continue working with Neuromaps repository and functionality. It is neccesary to implement spatial null models in statistical analysis of volumetric data. Documentation states that these are very computationally intensive, so I will be trying to find a way to run them in order to analyze maps in their native MNI152 space without transforming them. In doing so, it is unknown what amount of data or correlation is lost in translation. Additionally, I will be testing a few of these correlations with a different parcellation scheme. For a majority of the gene, AD matrix correlations, results were calculcated on left-hemisphere only after discarding rows with an AD value of 0, which left us with very little data to work with. I will be using a parcellation with 180+ parcels.

#### References
[1] Markello, RD, Hansen, JY, Liu, ZQ, Bazinet, V, Shafiei, G, Suarez, LE, Blostein, N, Seidlitz, J, Baillet, S, Satterthwaite, TD & Chakravarty, M. (2022). Neuromaps: structural and functional interpretation of brain maps. Biorxiv. doi:10.1101/bioRxiv.475081

[2] Hawrylycz MJ, Lein ES, Guillozet-Bongaarts AL, Shen EH, Ng L, Miller JA, van de Lagemaat LN, Smith KA, Ebbert A, Riley ZL, Abajian C, Beckmann CF, Bernard A, Bertagnolli D, Boe AF, Cartagena PM, Chakravarty MM, Chapin M, Chong J, Dalley RA, David Daly B, Dang C, Datta S, Dee N, Dolbeare TA, Faber V, Feng D, Fowler DR, Goldy J, Gregor BW, Haradon Z, Haynor DR, Hohmann JG, Horvath S, Howard RE, Jeromin A, Jochim JM, Kinnunen M, Lau C, Lazarz ET, Lee C, Lemon TA, Li L, Li Y, Morris JA, Overly CC, Parker PD, Parry SE, Reding M, Royall JJ, Schulkin J, Sequeira PA, Slaughterbeck CR, Smith SC, Sodt AJ, Sunkin SM, Swanson BE, Vawter MP, Williams D, Wohnoutka P, Zielke HR, Geschwind DH, Hof PR, Smith SM, Koch C, Grant SGN, Jones AR. An anatomically comprehensive atlas of the adult human brain transcriptome. Nature. 2012 Sep 20;489(7416):391-399. doi: 10.1038/nature11405. PMID: 22996553; PMCID: PMC4243026.

[2] Sipko van Dam, Urmo Võsa, Adriaan van der Graaf, Lude Franke, João Pedro de Magalhães, Gene co-expression analysis for functional classification and gene–disease predictions, Briefings in Bioinformatics, Volume 19, Issue 4, July 2018, Pages 575–592, https://doi.org/10.1093/bib/bbw139

[3] R.D. Markello, B. Misic (2021). Comparing spatial null models for brain map. Neuroimage, 236, p. 118052, 10.1016/j.neuroimage.2021.118052

[4] Teresa Gómez-Isla, Joseph L. Price, Daniel W. McKeel Jr., John C. Morris, John H. Growdon Bradley T. Hyman, Profound Loss of Layer II Entorhinal Cortex Neurons Occurs in Very Mild Alzheimer’s Disease, Journal of Neuroscience 15 July 1996, 16 (14) 4491-4500; DOI: https://doi.org/10.1523/JNEUROSCI.16-14-04491.1996

[5] Rui-ting Hu, Qian Yu, Shao-dan Zhou, Yi-xin Yin, Rui-guang Hu, Hai-peng Lu and Bang-li Hu (2020). Co-expression Network Analysis Reveals Novel Genes Underlying Alzheimer’s Disease Pathogenesis. Frontier Aging Neuroscience, https://doi.org/10.3389/fnagi.2020.605961

[6] Chun-I Sze, MD, Juan C. Troncoso, MD, Claudia Kawas, MD, Peter Mouton, PhD, Donald L. Price, MD, Lee J. Martin, PhD, Loss of the Presynaptic Vesicle Protein Synaptophysin in Hippocampus Correlates with Cognitive Decline in Alzheimer Disease, Journal of Neuropathology & Experimental Neurology, Volume 56, Issue 8, August 1997, Pages 933–944, https://doi.org/10.1097/00005072-199708000-00011

[7] Kril, J.J., Patel, S., Harding, A.J. et al. Neuron loss from the hippocampus of Alzheimer's disease exceeds extracellular neurofibrillary tangle formation. Acta Neuropathol 103, 370–376 (2002). https://doi.org/10.1007/s00401-001-0477-5

[8] Paul M Thompson, Kiralee M Hayashi, Greig I de Zubicaray, Andrew L Janke, Stephen E Rose, James Semple, Michael S Hong, David H Herman, David Gravano, David M Doddrell, Arthur W Toga, Mapping hippocampal and ventricular change in Alzheimer disease, NeuroImage, Volume 22, Issue 4, 2004, ISSN 1053-8119,https://doi.org/10.1016/j.neuroimage.2004.03.040.

[9] Scheff, Stephen W. et al. ‘Synaptic Loss in the Inferior Temporal Gyrus in Mild Cognitive Impairment and Alzheimer’s Disease’. 1 Jan. 2011 : 547 – 557.

[10] Manja Lehmann, Abdel Douiri, Lois G. Kim, Marc Modat, Dennis Chan, Sebastien Ourselin, Josephine Barnes, Nick C. Fox, Atrophy patterns in Alzheimer's disease and semantic dementia: A comparison of FreeSurfer and manual volumetric measurements, NeuroImage, Volume 49, Issue 3, 2010, ISSN 1053-8119, https://doi.org/10.1016/j.neuroimage.2009.10.056.

[11] Echávarri, C., Aalten, P., Uylings, H.B.M. et al. Atrophy in the parahippocampal gyrus as an early biomarker of Alzheimer’s disease. Brain Struct Funct 215, 265–271 (2011). https://doi.org/10.1007/s00429-010-0283-8

[12] Ross D. Markello, Aurina Arnatkevi, Jean-Baptiste Poline, Ben D. Fulcher, Alex Fornito, Bratislav Misic (2021), Standardizing workflows in imaging transcriptomics with the abagen toolbox. https://doi.org/10.1101/2021.07.08.451635

#### Links
* Neurosynth: https://neurosynth.org
* Access Allen Human Brain Atlas Gene Expression Data w/ abagen: https://abagen.readthedocs.io
* Check for autocorrelation in Python: https://scicoding.com/4-ways-of-calculating-autocorrelation-in-python/
* Neuromaps Setup & User Guide: https://netneurolab.github.io/neuromaps/index.html
* Information and recommendations for spatial nulls: https://markello-spatialnulls.netlify.app/index.html
* Project repo: https://github.com/voytekresearch/BIRD